In [ ]:
#Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#pip install pandas_profiling
#package produces an interactive set of tables and visualizations for exploratory data exploration(EDA)
from pandas_profiling import ProfileReport

pd.options.mode.chained_assignment = None  # default='warn' #To ignore the default warnings

In [ ]:
#Read the csv into df called data

data=pd.read_csv('Data_311.csv')

### Refining the case_enquiry_id column values to helpus uniquely identify the records.

In [ ]:
data['case_enquiry_id'] = data.reset_index().index + 1
data.head()

### Splitting timestamp column to separate date and time columns 

In [ ]:
data['open_date'] = pd.to_datetime(data['open_dt']).dt.date
data['open_time'] = pd.to_datetime(data['open_dt']).dt.time
data['target_date'] = pd.to_datetime(data['target_dt']).dt.date
data['target_time'] = pd.to_datetime(data['target_dt']).dt.time
data['closure_date'] = pd.to_datetime(data['closed_dt']).dt.date
data['closure_time'] = pd.to_datetime(data['closed_dt']).dt.time

In [ ]:
profile_CA = ProfileReport(df, title = "311 Data Profile")
profile_CA.to_file("311_profile_report.html")

# Graph 1: Case Status Analysis

In [ ]:
plt.figure(figsize=(10,6))
ax = data['case_status'].value_counts(normalize=True).plot(kind='bar', color=['orange', 'blue']) # specify bar colors and normalize data
plt.title('Case Status Analysis')
plt.xlabel('Open/Closed')
plt.ylabel('% of cases')
plt.xticks(rotation=0)

# add percentage labels to bars
for p in ax.patches:
    ax.annotate('{:.2f}%'.format(p.get_height()*100), (p.get_x()+0.2, p.get_height()+0.01))

plt.show()


# Graph 2: Case Resolution Analysis


In [ ]:
plt.figure(figsize=(10,6))
ax = data['ontime'].value_counts(normalize=True).plot(kind='bar', color=['orange', 'blue']) # specify bar colors and normalize data
plt.title('Case Resolution Analysis')
plt.xlabel('Ontime/Overdue')
plt.ylabel('% of cases closed')
plt.xticks(rotation=0)

# add percentage labels to bars
for p in ax.patches:
    ax.annotate('{:.2f}%'.format(p.get_height()*100), (p.get_x()+0.2, p.get_height()+0.01))

plt.show()


# Graph 3: Complain Frequency over the year 2022

In [ ]:

plt.figure(figsize=(10,6))
data.groupby('open_date')['case_title'].count().plot()
plt.title('Complain frequency over the months')
plt.xlabel('Date')
plt.ylabel('Number of complaints')
plt.show()


In [ ]:
plt.figure(figsize=(10,6))
data.groupby('open_date')['case_title'].count().plot()
plt.title('Complain frequency over the months')
plt.xlabel('Date')
plt.ylabel('Number of complaints')
plt.show()

## Graph 4

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(data['longitude'], data['latitude'])
plt.title('Location of complaints')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


# Load the cleaned csv

In [ ]:
data1=pd.read_csv('311_2022_Cleaned.csv')

# Correlation Matrix

In [ ]:
# Create correlation matrix
corr = data1.corr()

# Create heatmap of correlation matrix
plt.figure(figsize=(10,6))
sns.heatmap(corr, cmap='Blues', annot=True, linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.show()


## Graph 5

In [ ]:
plt.figure(figsize=(10,6))
complaints_by_dept = data1.groupby(['dept_responsible'])['case_title'].count().sort_values(ascending=False)
complaints_by_dept.plot(kind='bar', stacked=True)
plt.title('Number of complaints by department responsible')
plt.xlabel('Department')
plt.ylabel('Number of complaints')
plt.show()


In [ ]:
data_season = data1

### Creating a new column Season and categorizing ech records in either of the values based on 'open_date'

In [ ]:
import pandas as pd

# Convert the 'open_dt' column to a datetime object
data_season['open_date'] = pd.to_datetime(data_season['open_date'])

# Create a dictionary mapping each season to its start and end dates
seasons = {
    'Winter': {'start': pd.to_datetime('2022-01-01'), 'end': pd.to_datetime('2022-03-19')},
    'Spring': {'start': pd.to_datetime('2022-03-20'), 'end': pd.to_datetime('2022-06-20')},
    'Summer': {'start': pd.to_datetime('2022-06-21'), 'end': pd.to_datetime('2022-09-22')},
    'Fall': {'start': pd.to_datetime('2022-09-23'), 'end': pd.to_datetime('2022-12-31')}
}

# Define a function to assign the season based on the start date
def assign_season(row):
    for season, dates in seasons.items():
        if row['open_date'] >= dates['start'] and row['open_date'] <= dates['end']:
            return season

# Create a new column 'season' using the assign_season function
data_season['season'] = data_season.apply(assign_season, axis=1)


# Graph 6: Top 5 Complains per season


In [ ]:
season_team_responsible = data_season.groupby(['season', 'team_responsible']).size().unstack()

# Sort the values in descending order based on the sum of each column
season_team_responsible = season_team_responsible.loc[:, season_team_responsible.sum().sort_values(ascending=False).index]

# Select only the top 5 rows
season_team_responsible = season_team_responsible.iloc[:5]

# Select only the top 5 complaints for each season
top_complaints = season_team_responsible.apply(lambda x: x.sort_values(ascending=False).head(5), axis=1)

# Plot the graph
top_complaints.plot(kind='bar', stacked=True)
plt.xlabel('Season')
plt.ylabel('Frequency')
plt.title('Top 5 Complaint Frequencies by Season for Each Team')
plt.legend(title='Teams Responsible')
plt.show()
